# subscripts
set T;  # Time periods

set P;  # Products

set P'; # Finished products that can be sold

set I;  # Locations that produce products

set J;  # Customer regions

set K;  # Transportation modes

set C;  # Set of customer regions

set F;  # Set of locations that produce finished products

# parameters
param r{P,T};       # Revenue per unit of product in period

param c{I,J,K,T};   # Transportation cost

param v{I,P,T};     # Incremental processing cost

param h{P};         # Inventory holding cost

param a{I,P};       # Production capacity (time)

param alpha{P',P};  # Units of product required

param Production_Capacity{I,T};  # Production capacity

param d{J,P,T};     # Demand of product by customer region in period

# decision variables
var x{I,J,K,P,T} >= 0;  # Product transported

var y{I,P,T} >= 0;      # Product produced

var I{I,P,T} >= 0;      # Inventory

# Objective function
maximize Total_Profit:
  sum{t in T, p in P} r[p,t] * (sum{i in I, j in J, k in K} x[i,j,k,p,t])
  - sum{t in T, i in I, j in J, k in K, p in P} c[i,j,k,t] * x[i,j,k,p,t]
  - sum{t in T, i in I, p in P} v[i,p,t] * y[i,p,t]
  - sum{t in T, i in I, p in P} h[p] * I[i,p,t];

In [4]:
import pandas as pd
from  scipy.optimize import linprog as lp
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

In [9]:
!pip install openpyxl

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [10]:

"""
xls = pd.ExcelFile('/content/Datasets.xlsx')
Transportation = pd.read_excel(xls, 'Transportation')
Product = pd.read_excel(xls, 'Products')
Production_location = pd.read_excel(xls, 'production_location')
demand = pd.read_excel(xls, 'customer_demand')
"""
# Friendly reminder from Geoffrey
# Make sure you have the right path to acccess the files
# I made shortcuts to link the shared folder "IEOR 180 Team 6" to My drive and you can do the same to use the file path here

file_path = "Datasets.xlsx"
Transportation = pd.read_excel(file_path, sheet_name='Transportation')
Product = pd.read_excel(file_path, sheet_name='Products')
Production_location = pd.read_excel(file_path, sheet_name='production_location')
demand = pd.read_excel(file_path, sheet_name='customer_demand')

demand.head()

,customer_region,product,demand,time_t
0,C1,package_1,30,1
1,C1,package_2,50,1
2,C2,module_1,20,1
3,C2,package_2,10,1
4,C2,module_2,30,1


In [11]:
revenue_falling_rate = 0.1
holding_rate= 0.125 #annual holding rate is 50%, so per quater is 12.5%
total_time = 2

In [14]:
Product_initial = Product.loc[Product['Is_final_good'].notnull(), :].copy()
Product_initial["inventorycost_h"]=Product["revenue_r"]*holding_rate

In [15]:
###Product detail
product_weight = dict(zip(Product_initial['Product_p'], Product_initial['weight_w']))
product_revenue = dict(zip(Product_initial['Product_p'], Product_initial['revenue_r']))
product_alpha = dict(zip(Product_initial['Product_p'], Product_initial["alpha"]))
product_a = dict(zip(Product_initial['Product_p'], Product_initial["product_time_a"]))
product_cost= dict(zip(Product_initial['Product_p'], Product_initial['incrementalcost_v'])) ##assume the cost does not change


In [31]:
## assume the price decreases by revenue_falling_rate
revenue_over_t={}
for t in range(1, total_time+1):
  for key, revenue in product_revenue.items():
    new_key=(key, t)
    rt=revenue*(1-revenue_falling_rate)**t
    revenue_over_t[new_key]=rt


#### revenue_over_t, p,t, is the price of each product at time t
revenue_over_t

{('fab_1', 1): 9.0,
 ('fab_2', 1): 18.0,
 ('package_1', 1): 27.0,
 ('package_2', 1): 27.0,
 ('module_1', 1): 36.0,
 ('module_2', 1): 31.5,
 ('fab_1', 2): 8.100000000000001,
 ('fab_2', 2): 16.200000000000003,
 ('package_1', 2): 24.3,
 ('package_2', 2): 24.3,
 ('module_1', 2): 32.400000000000006,
 ('module_2', 2): 28.35}

In [33]:
Transportation=Transportation.merge(Production_location, left_on=['Location_i', 'Time_t'], right_on=['Location_i', 'time_t'])
data_tuples = list(Transportation.itertuples(index=False, name=None))
transport_goods={(*t[:4],): t[8] for t in data_tuples}
costs_weight = {(*t[:4],): t[5] for t in data_tuples}
ghgs_weight = {(*t[:4],): t[4] for t in data_tuples}

def weight_to_product(weight_data, product_weight=product_weight):
  # New dictionary for cost per product
  per_product = {}

  # Iterate through the cost_per_weight dictionary and multiply by product weight
  for key, cost in weight_data.items():
      # Extract the product name from your logic or data structure; assuming it's 'product_1' for all in this example
      product_name = transport_goods[key].split(', ')
      for i in product_name:
          #if i in product_weight:  # Check if the product name exists in the product_weight dictionary
          new_key = key + (i,)  # Create a new key for the cost_per_product dictionary
          weight = product_weight[i]  # Get the weight of the product
          per_product[new_key] = cost * weight  # Calculate cost per product and assign to the new dictionary
  return per_product


####Transportation Cost ijktp dictionary
cost_per_product=weight_to_product(costs_weight, product_weight)
####Transportation GHG Emission ijktp dictionary
ghgs_per_product=weight_to_product(ghgs_weight, product_weight)
ghgs_per_product

{('F1', 'P1', 'Flight', 1.0, 'fab_1'): 0.05,
 ('F1', 'P1', 'Flight', 1.0, 'fab_2'): 0.05,
 ('F1', 'P2', 'Flight', 1.0, 'fab_1'): 0.03,
 ('F1', 'P2', 'Flight', 1.0, 'fab_2'): 0.03,
 ('F2', 'P1', 'Truck', 1.0, 'fab_1'): 0.005,
 ('F2', 'P1', 'Truck', 1.0, 'fab_2'): 0.005,
 ('F2', 'P2', 'Flight', 1.0, 'fab_1'): 0.03,
 ('F2', 'P2', 'Flight', 1.0, 'fab_2'): 0.03,
 ('P1', 'M1', 'Flight', 1.0, 'package_1'): 0.04,
 ('P1', 'M1', 'Flight', 1.0, 'package_2'): 0.04,
 ('P1', 'M2', 'Flight', 1.0, 'package_1'): 0.1,
 ('P1', 'M2', 'Flight', 1.0, 'package_2'): 0.1,
 ('P2', 'M1', 'Flight', 1.0, 'package_1'): 0.12,
 ('P2', 'M1', 'Flight', 1.0, 'package_2'): 0.12,
 ('P2', 'M2', 'Truck', 1.0, 'package_1'): 0.016,
 ('P2', 'M2', 'Truck', 1.0, 'package_2'): 0.016,
 ('P1', 'C1', 'Flight', 1.0, 'package_1'): 0.16,
 ('P1', 'C1', 'Flight', 1.0, 'package_2'): 0.16,
 ('P1', 'C2', 'Flight', 1.0, 'package_1'): 0.14,
 ('P1', 'C2', 'Flight', 1.0, 'package_2'): 0.14,
 ('P2', 'C1', 'Flight', 1.0, 'package_1'): 0.08,
 ('P2

In [18]:
Production_location
data_tuples = list(Production_location.itertuples(index=False, name=None))
production_capacity_it={(*t[0: 2],): t[2] for t in data_tuples}
location_processtime_coe_itp={(t[0], t[1], fab.strip()): t[5] for t in data_tuples for fab in t[3].split(',')}
location_incrementalcost_coe_itp={(t[0], t[1], fab.strip()): t[4] for t in data_tuples for fab in t[3].split(',')}


In [19]:
### Varibale cost for each product at each location at time t
incrementalcost_itp={(i, t, p):
     product_cost[p]*location_incrementalcost_coe_itp[i, t, p] for i, t, p in  location_incrementalcost_coe_itp.keys()}
### Processing time dictionary for each product at each location at time t
processtime_itp={(i, t, p):
     product_a[p]*location_processtime_coe_itp[i, t, p] for i, t, p in  location_processtime_coe_itp.keys()}

### Two useful lists to track the value

In [20]:
ijktp_list=[i for i in cost_per_product.keys()]
itp_list =[i for i in location_incrementalcost_coe_itp.keys()]

### Usefull data dictionary:

#### r_pt the price for each product at time t
revenue_over_t
#### c_ijktp the transportation cost
cost_per_product 
#### g_ijktp the ghg emission cost
ghgs_per_product 
#### v_itp the varibale cost for one-unit product p at location i at time t
incrementalcost_itp 
#### a_itp the processing time for one-unit product p at location i at time t
processtime_itp 
#### product_alpha represents how many input products required to produce on product p
product_alpha 

### Decision Varaible

In [21]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 29.5 MB/s eta 0:00:0000:0100:01


In [22]:
import pandas as pd
import pulp



# Define your linear programming problem
lp = pulp.LpProblem("Maximize_revenue", pulp.LpMinimize)

# Decision variables
X = pulp.LpVariable.dicts("X",
                          [xijktp for xijktp in ijktp_list],
                          lowBound=0, cat='Continuous')
Y = pulp.LpVariable.dicts("Y",
                          [yitp for yitp in itp_list],
                          lowBound=0, cat='Continuous')
I = pulp.LpVariable.dicts("I",
                          [itp for itp in itp_list],
                          lowBound=0, cat='Continuous')




### Objective Functions

In [25]:
customer_region = demand['customer_region'].unique()
customer_region
## Objective function
lp += pulp.lpSum([revenue_over_t[p, t] *
                  pulp.lpSum([X[xijktp] for xijktp in ijktp_list if xijktp[1] in customer_region and xijktp[-1]==p and xijktp[-2]==t])
                  for p, t in revenue_over_t.keys()]) - \
      pulp.lpSum([cost_per_product[ijkpt]* X[ijkpt] for ijkpt in ijktp_list]) - \
      pulp.lpSum([incrementalcost_itp[itp]* Y[itp] for itp in itp_list]) -\
      pulp.lpSum([revenue_over_t[(p, t)]*holding_rate * I[i, t, p] for i, t, p in itp_list]), "Maximize_revenue"




### Constraints

In [24]:
# Constraints
for i, t, p in itp_list:
    t_plus_one = t + 1

    sum_x = pulp.lpSum([X.get((i2, j, k, tp, p2), 0) for (i2, j, k, tp, p2) in ijktp_list if i2 == i and tp == t_plus_one and p2 == p])


    inventory_next = I.get((i, t_plus_one, p), 0)
    inventory_current = I.get((i, t, p), 0)
    production_current = Y.get((i, t, p), 0)

    lp += inventory_next == inventory_current + production_current - sum_x

In [26]:
# Productiion capactiy constraint - 1

unique_products = set(p for (i, t, p) in processtime_itp.keys())

for i, t in production_capacity_it.keys():
    production_required = pulp.lpSum(processtime_itp.get((i, t, p), 0) * Y.get((i, p, t), 0) for p in unique_products)
    lp += production_required <= pulp.lpSum([X.get((i2, j, k, tp, p2), 0) for (i2, j, k, tp, p2) in ijktp_list])

# Production capacity constraint - 2

for i, t in production_capacity_it.keys():
    production_sum = pulp.lpSum(processtime_itp.get((i, t, p), 0) * Y.get((i, p, t), 0) for p in unique_products)
    lp += production_sum <= production_capacity_it[(i, t)]



In [27]:
# Demand Constraint

for index, row in demand.iterrows():
    product = row['product']  # p
    customer_region = row['customer_region']  # c
    period = row['time_t']  # t'
    demand_quantity = row['demand']  # d_{p,c,t'}

    relevant_variables = [X[(i, j, k, p, t)] for (i, j, k, p, t) in X.keys() if j == customer_region and p == product and t <= period]
    lp += pulp.lpSum(relevant_variables) <= demand_quantity


In [28]:
# Greenhouse Gases Constraint

GHG_Lim = 10000
ghg_emissions_total = pulp.lpSum([ghgs_per_product[(i, j, k, t, p)] * X[(i, j, k, t, p)]
                                  for (i, j, k, t, p) in X.keys()
                                  if (i, j, k, t, p) in ghgs_per_product])

lp += ghg_emissions_total <= GHG_Lim

In [29]:
# Non-negativity constraints
for var in Y.values():
    lp += var >= 0
for var in X.values():
    lp += var >= 0
for var in I.values():
    lp += var >= 0

In [ ]:
#!apt-get install -y glpk-utils

In [30]:

lp.solve()
print("Status:", pulp.LpStatus[lp.status])
print("Objective Value:", pulp.value(lp.objective))


for variable in lp.variables():
    if variable.varValue > 0:
        print(f"{variable.name} = {variable.varValue}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /srv/conda/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/4a4f5d626ef24ec4b5c3f7371df3a733-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/4a4f5d626ef24ec4b5c3f7371df3a733-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 200 COLUMNS
At line 1441 RHS
At line 1637 BOUNDS
At line 1638 ENDATA
Problem MODEL has 195 rows, 112 columns and 1128 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-195) rows, 0 (-112) columns and 0 (-1128) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value -6002000
After Postsolve, objective -6002000, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -6002000 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seco